In [1]:
import sys
import os
import torch
import matplotlib.pyplot as plt
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

In [3]:
from bengali_asr.models import Whisper
from configs.whisper_characterwise import Configs
CFG = Configs()

length of train: 200, length of valid: 200


In [5]:
from trainer.trainer import Trainer

In [6]:
trainer = Trainer(CFG)

torch.Size([1, 86, 80])